In [ ]:
!pip install pandas

In [29]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/train.csv')

In [42]:
# Count occurrences of `ImageId` in df
df['count'] = df.ImageId.apply(lambda x: df['ImageId'].value_counts()[x])

In [41]:
df.head(10)

,ImageId,ClassId,EncodedPixels,count
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,1
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...,1
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,1
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...,1
5,0025bde0c.jpg,3,8458 14 8707 35 8963 48 9219 71 9475 88 9731 8...,2
6,0025bde0c.jpg,4,315139 8 315395 15 315651 16 315906 17 316162 ...,2
7,002af848d.jpg,4,290800 6 291055 13 291311 15 291566 18 291822 ...,1
8,002fc4e19.jpg,1,146021 3 146275 10 146529 40 146783 46 147038 ...,2
9,002fc4e19.jpg,2,145658 7 145901 20 146144 33 146386 47 146629 ...,2


In [53]:
df_duplicates = df.query('count > 1')
df_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 856 entries, 5 to 7082
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ImageId        856 non-null    object
 1   ClassId        856 non-null    int64 
 2   EncodedPixels  856 non-null    object
 3   count          856 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 33.4+ KB


In [7]:
df.groupby('ClassId').count()

,ImageId,EncodedPixels
ClassId,,
1,897,897
2,247,247
3,5150,5150
4,801,801


--> there is a strong **imbalance** in the representation of different defect type:
- error 1 = 0.135 %
- error 2 = 0.037 %
- error 3 = 0.773 % of all images
- error 4 = 0.120 %

In [58]:
# Dimension of our pictures
1600*256

409600

In [57]:
print(f"The training data has {df.ImageId.nunique()} unique image IDs")

The training data has 6666 unique image IDs


### To-Dos Daniela

In [ ]:
pd.hist(df_duplicates.count)

In [ ]:
pd.pivot_table(df_duplicates, index='ImageId', columns='ClassId', values='ImageId', aggfunc = 'count', fill_value=0)
